<a href="https://colab.research.google.com/github/thakkar-hiren/News-Summarisation/blob/main/2_News_Summary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Text Summarisation using Bigram overlapping and Text Rank Algorithm:
This file contains an implementation of a text summarization technique utilizing the PageRank algorithm, a methodology famously employed by search engines like Google. The algorithm ranks sentences within a text based on their perceived importance and selects the top-ranked sentences to compose a concise summary. The process begins by loading a dataset containing news articles, preprocessing the text, and then applying the summarization algorithm to generate summaries of varying lengths (3 and 5 sentences). One of the key steps involves computing the similarity between sentences using the Jaccard similarity coefficient. This coefficient is calculated by determining the intersection of bi-grams (sets of two words) between two sentences and dividing it by the union of their bi-grams. This similarity measure assists in gauging the degree of overlap or similarity between the bi-gram sets of two sentences, aiding in the ranking process. The quality of the generated summaries is evaluated using the ROUGE (Recall-Oriented Understudy for Gisting Evaluation) metric, which compares them to human-generated reference summaries. Finally, the file outputs the average ROUGE scores, offering insights into the effectiveness of the summarization technique. This implementation finds applications in various domains such as document summarization, information retrieval, and text mining.



# Mounting Drive

In [ ]:
# Mounting Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Loading Data

In [ ]:
# Loading Dataset
import pandas as pd
news_data = pd.read_csv('/content/drive/MyDrive/Datasets/filtered_news_data.csv')
news_data

,Unnamed: 0,text,ctext,text_len,ctext_len,text_sent_count,text_word_count,ctext_sent_count,ctext_word_count,preprocessed_text,preprocessed_ctext
0,0,The Administration of Union Territory Daman an...,The Daman and Diu administration on Wednesday ...,358,2313,2,62,16,413,the administration of union territory daman an...,the daman and diu administration on wednesday ...
1,2,The Indira Gandhi Institute of Medical Science...,The Indira Gandhi Institute of Medical Science...,398,2112,3,70,18,379,the indira gandhi institute of medical science...,the indira gandhi institute of medical science...
2,4,Hotels in Maharashtra will train their staff t...,Hotels in Mumbai and other Indian cities are t...,366,3249,3,67,8,569,hotels in maharashtra will train their staff t...,hotels in mumbai and other indian cities are t...
3,5,A 32-year-old man on Wednesday was found hangi...,An alleged suspect in a kidnapping case was fo...,347,2247,3,63,23,440,a 32yearold man on wednesday was found hanging...,an alleged suspect in a kidnapping case was fo...
4,6,The Delhi High Court reduced the compensation ...,"In an interesting ruling, the Delhi high court...",361,2367,3,70,11,444,the delhi high court reduced the compensation ...,in an interesting ruling the delhi high court ...
...,...,...,...,...,...,...,...,...,...,...,...
3911,4507,After emerging as the single largest party in ...,As uncertainty loomed large in the BMC polls d...,335,606,3,66,1,115,after emerging as the single largest party in ...,as uncertainty loomed large in the bmc polls d...
3912,4509,Fruit juice concentrate maker Rasna is eyeing ...,"Mumbai, Feb 23 (PTI) Fruit juice concentrate m...",361,1000,3,70,8,200,fruit juice concentrate maker rasna is eyeing ...,mumbai feb 23 pti fruit juice concentrate make...
3913,4510,Former Indian cricketer Sachin Tendulkar atten...,Former cricketer Sachin Tendulkar was spotted ...,362,933,2,63,3,158,former indian cricketer sachin tendulkar atten...,former cricketer sachin tendulkar was spotted ...
3914,4512,The Maharashtra government has initiated an in...,Maharahstra Power Minister Chandrashekhar Bawa...,387,1397,3,63,9,233,the maharashtra government has initiated an in...,maharahstra power minister chandrashekhar bawa...


# Building Model

In [ ]:
import nltk
from nltk.util import ngrams
from nltk.tokenize import word_tokenize, sent_tokenize
nltk.download('punkt')

def summarize_text(text, nos=5):
  # Initialization
  word_grams = []
  sent_to_idx = {}

  # Retrieving sentences from text using sent-tokenize
  sentences = [sentence for sentence in sent_tokenize(text) if len(sentence.split()) >= 10]

  for i in range(len(sentences)):
    sent_to_idx[sentences[i]] = i

  # print(sent_to_idx)

  # Tokenizing sentence. It will be helpfull in generating ngrams
  tokenized_sentences = [word_tokenize(sent) for sent in sentences]
  # print(sentences)
  # print(len(sentences))

  # Generating Bi-Grams using the tokenized sentences generated above
  for sentence in tokenized_sentences:
    word_grams.append([set(ngram) for ngram in ngrams(sentence, 2, pad_right=True, right_pad_symbol=' ')])

  # print(word_grams)
  # print(len(word_grams))

  # Creating Similarity Matrix...
  similarity_matrix = [[0 for _ in range(len(sentences))] for _ in range(len(sentences))]
  for i, sentence1 in enumerate(sentences):
    for j, sentence2 in enumerate(sentences):
      if i != j:
        intersection = len([value for value in word_grams[0] if value in word_grams[1]])
        similarity_matrix[i][j] = intersection / (len(word_grams[i]) + len(word_grams[j]) - intersection)

  # Implementing PageRank algorithm
  damping_factor = 0.85
  ranks = [1.0 for _ in range(len(sentences))]
  for _ in range(1500):  # Iterate 100 times for convergence
    new_ranks = [damping_factor * sum(old_rank * similarity for old_rank, similarity in zip(ranks, similarity_row)) + (1 - damping_factor) / len(sentences) for similarity_row in similarity_matrix]
    ranks = new_ranks

  # Select top-ranked sentences
  sorted_sentences = sorted(zip(sentences, ranks, range(len(sentences))), key=lambda x: x[1], reverse=True)[:nos]
  # print(sorted_sentences)

  ordered_sentences_id = []
  ordered_sentences = []

  # Getting the ids of top ranked sentences
  for sentence, _, _ in sorted_sentences:
    ordered_sentences_id.append(sent_to_idx[sentence])

  # print(ordered_sentences_id)

  # sorting the ordered_sentence_id
  ordered_sentences_id.sort()

  # Retrieving top rank sentences in the increasing order of thier ids.
  for id in ordered_sentences_id:
    ordered_sentences.append(sentences[id])

  # print(ordered_sentences)

  # Joining the top ranked sentences in the order they appeared in original article
  summary = " ".join(ordered_sentences)

  return summary

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
# Generating summaries for news_data with 5 sentences
news_data['predictedSummary_5'] = news_data['preprocessed_ctext'].apply(lambda x: summarize_text(x))
news_data

,Unnamed: 0,text,ctext,text_len,ctext_len,text_sent_count,text_word_count,ctext_sent_count,ctext_word_count,preprocessed_text,preprocessed_ctext,predictedSummary_5
0,0,The Administration of Union Territory Daman an...,The Daman and Diu administration on Wednesday ...,358,2313,2,62,16,413,the administration of union territory daman an...,the daman and diu administration on wednesday ...,it has been decided to celebrate the festival ...
1,2,The Indira Gandhi Institute of Medical Science...,The Indira Gandhi Institute of Medical Science...,398,2112,3,70,18,379,the indira gandhi institute of medical science...,the indira gandhi institute of medical science...,dr biswas had just returned after a fourday le...
2,4,Hotels in Maharashtra will train their staff t...,Hotels in Mumbai and other Indian cities are t...,366,3249,3,67,8,569,hotels in maharashtra will train their staff t...,hotels in mumbai and other indian cities are t...,we have hospitality professionals working in h...
3,5,A 32-year-old man on Wednesday was found hangi...,An alleged suspect in a kidnapping case was fo...,347,2247,3,63,23,440,a 32yearold man on wednesday was found hanging...,an alleged suspect in a kidnapping case was fo...,the police claimed it was an unprovoked suicid...
4,6,The Delhi High Court reduced the compensation ...,"In an interesting ruling, the Delhi high court...",361,2367,3,70,11,444,the delhi high court reduced the compensation ...,in an interesting ruling the delhi high court ...,while he was crossing the road he was hit and ...
...,...,...,...,...,...,...,...,...,...,...,...,...
3911,4507,After emerging as the single largest party in ...,As uncertainty loomed large in the BMC polls d...,335,606,3,66,1,115,after emerging as the single largest party in ...,as uncertainty loomed large in the bmc polls d...,as uncertainty loomed large in the bmc polls d...
3912,4509,Fruit juice concentrate maker Rasna is eyeing ...,"Mumbai, Feb 23 (PTI) Fruit juice concentrate m...",361,1000,3,70,8,200,fruit juice concentrate maker rasna is eyeing ...,mumbai feb 23 pti fruit juice concentrate make...,we are looking at rs 250 crore from this categ...
3913,4510,Former Indian cricketer Sachin Tendulkar atten...,Former cricketer Sachin Tendulkar was spotted ...,362,933,2,63,3,158,former indian cricketer sachin tendulkar atten...,former cricketer sachin tendulkar was spotted ...,although sachin attended rajya sabha proceedin...
3914,4512,The Maharashtra government has initiated an in...,Maharahstra Power Minister Chandrashekhar Bawa...,387,1397,3,63,9,233,the maharashtra government has initiated an in...,maharahstra power minister chandrashekhar bawa...,bhosle was stumped after receiving an electric...


In [ ]:
# Generating summaries for news_data with 3 sentences
news_data['predictedSummary_3'] = news_data['preprocessed_ctext'].apply(lambda x: summarize_text(x,nos=3))
news_data

,Unnamed: 0,text,ctext,text_len,ctext_len,text_sent_count,text_word_count,ctext_sent_count,ctext_word_count,preprocessed_text,preprocessed_ctext,predictedSummary_5,predictedSummary_3
0,0,The Administration of Union Territory Daman an...,The Daman and Diu administration on Wednesday ...,358,2313,2,62,16,413,the administration of union territory daman an...,the daman and diu administration on wednesday ...,it has been decided to celebrate the festival ...,it has been decided to celebrate the festival ...
1,2,The Indira Gandhi Institute of Medical Science...,The Indira Gandhi Institute of Medical Science...,398,2112,3,70,18,379,the indira gandhi institute of medical science...,the indira gandhi institute of medical science...,dr biswas had just returned after a fourday le...,dr biswas had just returned after a fourday le...
2,4,Hotels in Maharashtra will train their staff t...,Hotels in Mumbai and other Indian cities are t...,366,3249,3,67,8,569,hotels in maharashtra will train their staff t...,hotels in mumbai and other indian cities are t...,we have hospitality professionals working in h...,we have hospitality professionals working in h...
3,5,A 32-year-old man on Wednesday was found hangi...,An alleged suspect in a kidnapping case was fo...,347,2247,3,63,23,440,a 32yearold man on wednesday was found hanging...,an alleged suspect in a kidnapping case was fo...,the police claimed it was an unprovoked suicid...,the relatives of the deceased suspect foul pla...
4,6,The Delhi High Court reduced the compensation ...,"In an interesting ruling, the Delhi high court...",361,2367,3,70,11,444,the delhi high court reduced the compensation ...,in an interesting ruling the delhi high court ...,while he was crossing the road he was hit and ...,while he was crossing the road he was hit and ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3911,4507,After emerging as the single largest party in ...,As uncertainty loomed large in the BMC polls d...,335,606,3,66,1,115,after emerging as the single largest party in ...,as uncertainty loomed large in the bmc polls d...,as uncertainty loomed large in the bmc polls d...,as uncertainty loomed large in the bmc polls d...
3912,4509,Fruit juice concentrate maker Rasna is eyeing ...,"Mumbai, Feb 23 (PTI) Fruit juice concentrate m...",361,1000,3,70,8,200,fruit juice concentrate maker rasna is eyeing ...,mumbai feb 23 pti fruit juice concentrate make...,we are looking at rs 250 crore from this categ...,we are looking at taking it to the neighbourin...
3913,4510,Former Indian cricketer Sachin Tendulkar atten...,Former cricketer Sachin Tendulkar was spotted ...,362,933,2,63,3,158,former indian cricketer sachin tendulkar atten...,former cricketer sachin tendulkar was spotted ...,although sachin attended rajya sabha proceedin...,although sachin attended rajya sabha proceedin...
3914,4512,The Maharashtra government has initiated an in...,Maharahstra Power Minister Chandrashekhar Bawa...,387,1397,3,63,9,233,the maharashtra government has initiated an in...,maharahstra power minister chandrashekhar bawa...,bhosle was stumped after receiving an electric...,but the government has not bothered to take an...


# Model Evaluation

In [ ]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=3541a5dd7c135f861bcc2b6768e3da31c30c0aba9e20e36546c38729b6f088bc
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [ ]:
# Function to evaluate summaries generated with 3 sentences
from rouge_score import rouge_scorer
import numpy as np

def evaluate_summaries_3(news_data):
    scorer = rouge_scorer.RougeScorer(rouge_types=['rouge1', 'rouge2', 'rougeL'])
    rouge_scores = []

    for idx, row in news_data.iterrows():
        scores = scorer.score(target=row['preprocessed_text'], prediction=row['predictedSummary_3'])
        rouge_scores.append(scores)

    avg_rouge1_precision = np.mean([score['rouge1'].precision for score in rouge_scores])
    avg_rouge1_recall = np.mean([score['rouge1'].recall for score in rouge_scores])
    avg_rouge1_f1 = np.mean([score['rouge1'].fmeasure for score in rouge_scores])

    avg_rouge2_precision = np.mean([score['rouge2'].precision for score in rouge_scores])
    avg_rouge2_recall = np.mean([score['rouge2'].recall for score in rouge_scores])
    avg_rouge2_f1 = np.mean([score['rouge2'].fmeasure for score in rouge_scores])

    avg_rougeL_precision = np.mean([score['rougeL'].precision for score in rouge_scores])
    avg_rougeL_recall = np.mean([score['rougeL'].recall for score in rouge_scores])
    avg_rougeL_f1 = np.mean([score['rougeL'].fmeasure for score in rouge_scores])

    print("Average ROUGE-1 Precision: ", avg_rouge1_precision)
    print("Average ROUGE-1 Recall: ", avg_rouge1_recall)
    print("Average ROUGE-1 F1-Score: ", avg_rouge1_f1)
    print("Average ROUGE-2 Precision: ", avg_rouge2_precision)
    print("Average ROUGE-2 Recall: ", avg_rouge2_recall)
    print("Average ROUGE-2 F1-Score: ", avg_rouge2_f1)
    print("Average ROUGE-L Precision: ", avg_rougeL_precision)
    print("Average ROUGE-L Recall: ", avg_rougeL_recall)
    print("Average ROUGE-L F1-Score: ", avg_rougeL_f1)

In [ ]:
# Function to evaluate summaries generated with 5 sentences
def evaluate_summaries_5(news_data):
    scorer = rouge_scorer.RougeScorer(rouge_types=['rouge1', 'rouge2', 'rougeL'])
    rouge_scores = []

    for idx, row in news_data.iterrows():
        scores = scorer.score(target=row['preprocessed_text'], prediction=row['predictedSummary_5'])
        rouge_scores.append(scores)

    avg_rouge1_precision = np.mean([score['rouge1'].precision for score in rouge_scores])
    avg_rouge1_recall = np.mean([score['rouge1'].recall for score in rouge_scores])
    avg_rouge1_f1 = np.mean([score['rouge1'].fmeasure for score in rouge_scores])

    avg_rouge2_precision = np.mean([score['rouge2'].precision for score in rouge_scores])
    avg_rouge2_recall = np.mean([score['rouge2'].recall for score in rouge_scores])
    avg_rouge2_f1 = np.mean([score['rouge2'].fmeasure for score in rouge_scores])

    avg_rougeL_precision = np.mean([score['rougeL'].precision for score in rouge_scores])
    avg_rougeL_recall = np.mean([score['rougeL'].recall for score in rouge_scores])
    avg_rougeL_f1 = np.mean([score['rougeL'].fmeasure for score in rouge_scores])

    print("Average ROUGE-1 Precision: ", avg_rouge1_precision)
    print("Average ROUGE-1 Recall: ", avg_rouge1_recall)
    print("Average ROUGE-1 F1-Score: ", avg_rouge1_f1)
    print("Average ROUGE-2 Precision: ", avg_rouge2_precision)
    print("Average ROUGE-2 Recall: ", avg_rouge2_recall)
    print("Average ROUGE-2 F1-Score: ", avg_rouge2_f1)
    print("Average ROUGE-L Precision: ", avg_rougeL_precision)
    print("Average ROUGE-L Recall: ", avg_rougeL_recall)
    print("Average ROUGE-L F1-Score: ", avg_rougeL_f1)

In [ ]:
# Evaluation for summaries of 3 sentences
print("For Bigrams:\n ")
print("Evaluation for the summary of 3 sentences: \n")
evaluate_summaries_3(news_data)
print("\n")

# Evaluation for summaries of 5 sentences
print("Evaluation for the summary of 5 sentences: \n")
evaluate_summaries_5(news_data)

For Bigrams:
 
Evaluation for the summary of 3 sentences: 

Average ROUGE-1 Precision:  0.37414666475512076
Average ROUGE-1 Recall:  0.29368491992451473
Average ROUGE-1 F1-Score:  0.31945058835779594
Average ROUGE-2 Precision:  0.1398361305364862
Average ROUGE-2 Recall:  0.11638877314388543
Average ROUGE-2 F1-Score:  0.1227485737685749
Average ROUGE-L Precision:  0.255766713069067
Average ROUGE-L Recall:  0.20129471251228037
Average ROUGE-L F1-Score:  0.21851125505293273


Evaluation for the summary of 5 sentences: 

Average ROUGE-1 Precision:  0.3277822117496977
Average ROUGE-1 Recall:  0.4502218241629471
Average ROUGE-1 F1-Score:  0.3693910075222978
Average ROUGE-2 Precision:  0.13679659762968593
Average ROUGE-2 Recall:  0.1970688670855932
Average ROUGE-2 F1-Score:  0.15657227276691346
Average ROUGE-L Precision:  0.21693228479278864
Average ROUGE-L Recall:  0.30069901882764444
Average ROUGE-L F1-Score:  0.24490146378716615


# Rough Work

In [ ]:
# Evaluating Summary using ROUGE Score
from rouge_score import rouge_scorer
scorer = rouge_scorer.RougeScorer(rouge_types=['rouge1','rouge2','rougeL'])
rouge_scores = []

for idx, row in news_data.iterrows():
  scores = scorer.score(target=row['preprocessed_text'],prediction=row['predictedSummary'])
  rouge_scores.append(scores)

In [ ]:
print("Average ROUGE-1 Precision: ", sum(score['rouge1'].precision for score in rouge_scores)/len(rouge_scores))
print("Average ROUGE-1 Recall: ", sum(score['rouge1'].recall for score in rouge_scores)/len(rouge_scores))
print("Average ROUGE-1 F1-Score: ", sum(score['rouge1'].fmeasure for score in rouge_scores)/len(rouge_scores))
print("Average ROUGE-2 Precision: ", sum(score['rouge2'].precision for score in rouge_scores)/len(rouge_scores))
print("Average ROUGE-2 Recall: ", sum(score['rouge2'].recall for score in rouge_scores)/len(rouge_scores))
print("Average ROUGE-2 F1-Score: ", sum(score['rouge2'].fmeasure for score in rouge_scores)/len(rouge_scores))
print("Average ROUGE-L Precision: ", sum(score['rougeL'].precision for score in rouge_scores)/len(rouge_scores))
print("Average ROUGE-L Recall: ", sum(score['rougeL'].recall for score in rouge_scores)/len(rouge_scores))
print("Average ROUGE-L F1-Score: ", sum(score['rougeL'].fmeasure for score in rouge_scores)/len(rouge_scores))

Average ROUGE-1 Precision:  0.32778221174969746
Average ROUGE-1 Recall:  0.450221824162947
Average ROUGE-1 F1-Score:  0.36939100752229775
Average ROUGE-2 Precision:  0.13679659762968605
Average ROUGE-2 Recall:  0.19706886708559337
Average ROUGE-2 F1-Score:  0.156572272766914
Average ROUGE-L Precision:  0.2169322847927886
Average ROUGE-L Recall:  0.30069901882764405
Average ROUGE-L F1-Score:  0.24490146378716626


In [ ]:
news_data['preprocessed_text'][4]

'the delhi high court reduced the compensation awarded to a motor accident victim by 45 after it found negligence on the part of both parties . a compensation of 10 lakh was earlier awarded to the victim . the court observed its possible despite the vehicle being driven in permissible limit an accident can occur when a jaywalker suddenly appears on road .'

In [ ]:
news_data['predictedSummary_word_count'] = news_data['predictedSummary'].apply(lambda x: len(word_tokenize(x)))

In [ ]:
news_data

,Unnamed: 0,text,ctext,text_len,ctext_len,text_sent_count,text_word_count,ctext_sent_count,ctext_word_count,preprocessed_text,preprocessed_ctext,predictedSummary,predictedSummary_word_count
0,0,The Administration of Union Territory Daman an...,The Daman and Diu administration on Wednesday ...,358,2313,2,62,16,413,the administration of union territory daman an...,the daman and diu administration on wednesday ...,it has been decided to celebrate the festival ...,83
1,2,The Indira Gandhi Institute of Medical Science...,The Indira Gandhi Institute of Medical Science...,398,2112,3,70,18,379,the indira gandhi institute of medical science...,the indira gandhi institute of medical science...,dr biswas had just returned after a fourday le...,72
2,4,Hotels in Maharashtra will train their staff t...,Hotels in Mumbai and other Indian cities are t...,366,3249,3,67,8,569,hotels in maharashtra will train their staff t...,hotels in mumbai and other indian cities are t...,we have hospitality professionals working in h...,90
3,5,A 32-year-old man on Wednesday was found hangi...,An alleged suspect in a kidnapping case was fo...,347,2247,3,63,23,440,a 32yearold man on wednesday was found hanging...,an alleged suspect in a kidnapping case was fo...,the police claimed it was an unprovoked suicid...,64
4,6,The Delhi High Court reduced the compensation ...,"In an interesting ruling, the Delhi high court...",361,2367,3,70,11,444,the delhi high court reduced the compensation ...,in an interesting ruling the delhi high court ...,while he was crossing the road he was hit and ...,84
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3911,4507,After emerging as the single largest party in ...,As uncertainty loomed large in the BMC polls d...,335,606,3,66,1,115,after emerging as the single largest party in ...,as uncertainty loomed large in the bmc polls d...,as uncertainty loomed large in the bmc polls d...,111
3912,4509,Fruit juice concentrate maker Rasna is eyeing ...,"Mumbai, Feb 23 (PTI) Fruit juice concentrate m...",361,1000,3,70,8,200,fruit juice concentrate maker rasna is eyeing ...,mumbai feb 23 pti fruit juice concentrate make...,we are looking at rs 250 crore from this categ...,101
3913,4510,Former Indian cricketer Sachin Tendulkar atten...,Former cricketer Sachin Tendulkar was spotted ...,362,933,2,63,3,158,former indian cricketer sachin tendulkar atten...,former cricketer sachin tendulkar was spotted ...,although sachin attended rajya sabha proceedin...,86
3914,4512,The Maharashtra government has initiated an in...,Maharahstra Power Minister Chandrashekhar Bawa...,387,1397,3,63,9,233,the maharashtra government has initiated an in...,maharahstra power minister chandrashekhar bawa...,bhosle was stumped after receiving an electric...,81


In [ ]:
summarize_text(news_data['preprocessed_ctext'][4])

{'in an interesting ruling the delhi high court reduced the compensation awarded to a motor accident victim by 45 per cent after it found negligence on the part of both the parties .': 0, 'deepak kumar the victim was hit by a motor vehicle driven by a government employee named ajay kumar in palam .': 1, 'deepak told the court that while he was on his way to work he stopped his vehicle after crossing palam flyover to feed a cow .': 2, 'while he was crossing the road he was hit and suffered injuries .': 3, 'the trial court had awarded a compensation of rs 10 lakh to him after ruling negligence on ajays part .': 4, 'ajay challenged the trial court order based on the detailed accident report that mentioned motorcycle was not being driven dangerously or at an excessive speed and was being driven in its lane and within the permissible speed limit .': 5, 'he sought examination of the contributory negligence on behalf of the injured through advocate trideep pais and pranav jain .': 6, 'the cou

'while he was crossing the road he was hit and suffered injuries . the trial court had awarded a compensation of rs 10 lakh to him after ruling negligence on ajays part . he sought examination of the contributory negligence on behalf of the injured through advocate trideep pais and pranav jain . it exhibits his negligence towards his own safety the court added . in the circumstances a portion of the blame and negligence would have to be shared by the claimant .'